In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import t,ttest_ind,ks_2samp,shapiro,anderson 
from statsmodels.stats.weightstats import ztest
import statsmodels.api as sm
from scipy.stats import levene,bartlett,shapiro
import statsmodels.formula.api as smf

import scipy.stats as stats

warnings.filterwarnings("ignore")
import os
os.getcwd()

'C:\\Users\\Vahid\\OneDrive - University of Toronto\\MMA - 2022 -\\Jupyter files'

# Reading File

In [2]:
#path = '/Users/Vahid/Library/CloudStorage/OneDrive-UniversityofToronto/MMA - 2022 -/Jupyter files/'
path = "C:/Users/Vahid/OneDrive - University of Toronto/MMA - 2022 -/Jupyter files/"
df_raw = pd.read_excel(path+'Data/UBER.xlsx', sheet_name ='Switchbacks' )

In [3]:
df_raw.head()

,city_id,period_start,wait_time,treat,commute,trips_pool,trips_express,rider_cancellations,total_driver_payout,total_matches,total_double_matches
0,Boston,2018-02-19 07:00:00,2 mins,False,True,1415,3245,256,34458.411634,3372,1476
1,Boston,2018-02-19 09:40:00,5 mins,True,False,1461,2363,203,29764.349821,2288,1275
2,Boston,2018-02-19 12:20:00,2 mins,False,False,1362,2184,118,27437.367363,2283,962
3,Boston,2018-02-19 15:00:00,5 mins,True,True,1984,3584,355,44995.452993,4035,2021
4,Boston,2018-02-19 17:40:00,2 mins,False,False,1371,2580,181,27583.955295,2200,979


In [4]:
df_raw['total_ride'] = df_raw['trips_express'] + df_raw['trips_pool']
df_raw['cost_per_ride'] = df_raw['total_driver_payout']/df_raw['total_ride']
df_raw['match_rate'] = df_raw['total_matches']/df_raw['total_ride']
df_raw['double_matches_rate'] = df_raw['total_double_matches']/df_raw['total_ride']

## Question 1: What is the difference between ride cancellations and cost per trip of 2 min wait time against a 5min wait time? Is the difference statistically significant at a 5% confidence level? Write one-sided hypothesis tests and use parametric test statistics (you don’t need to validate the assumption of the parametric approach).


# One-Sided Hypothesis Test

### Summary Table

In [5]:
summary = df_raw.groupby(['wait_time'])['rider_cancellations','trips_pool','trips_express','total_ride','cost_per_ride','match_rate','double_matches_rate'].agg('mean').T
summary['diff (5mins - 2mins)'] = summary['5 mins'] - summary['2 mins']
summary

wait_time,2 mins,5 mins,diff (5mins - 2mins)
rider_cancellations,165.349206,190.126984,24.777778
trips_pool,1355.317460,1460.698413,105.380952
trips_express,2611.666667,2419.380952,-192.285714
total_ride,3966.984127,3880.079365,-86.904762
cost_per_ride,7.364866,6.987458,-0.377409
match_rate,0.659115,0.624274,-0.034842
double_matches_rate,0.321761,0.348986,0.027224


## Ride Cancellation

In [6]:
statistic,p_value =  ttest_ind(df_raw[df_raw['wait_time']=='2 mins']['rider_cancellations'], df_raw[df_raw['wait_time']=='5 mins']['rider_cancellations'], alternative='less')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))


Statistics = -2.764, P-value=0.003


# Cost Per Rider

In [7]:
statistic,p_value =  ttest_ind(df_raw[df_raw['wait_time']=='2 mins']['cost_per_ride'], df_raw[df_raw['wait_time']=='5 mins']['cost_per_ride'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))


Statistics = 3.462, P-value=0.000


## Question 2: Does the total number of express pool trips,  ride cancellations, and cost per trip change for commuting versus non-commuting hours at a 5% level of confidence? Use a one-sided hypothesis test to confirm your findings (Use only non-parametric test statistics)

### Commute Hours

In [8]:
df_commute = df_raw[df_raw['commute']==True]

In [13]:
summary = df_commute.groupby(['wait_time'])['rider_cancellations','trips_pool','trips_express','total_ride','cost_per_ride','match_rate','double_matches_rate'].agg('median').T
summary['diff (5mins - 2mins)'] = summary['5 mins'] - summary['2 mins']
summary.round(2)

wait_time,2 mins,5 mins,diff (5mins - 2mins)
rider_cancellations,240.50,290.00,49.50
trips_pool,1478.00,1618.50,140.50
trips_express,3434.50,3280.50,-154.00
total_ride,4859.50,4694.00,-165.50
cost_per_ride,7.99,7.65,-0.34
match_rate,0.76,0.75,-0.01
double_matches_rate,0.35,0.36,0.01


### Average Ride Cancellation

In [16]:
statistic,p_value =  stats.mannwhitneyu(df_commute[df_commute['wait_time']=='2 mins']['rider_cancellations'], df_commute[df_commute['wait_time']=='5 mins']['rider_cancellations'], alternative='less')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))

Statistics = 13.000, P-value=0.003


### Cost per trip

In [17]:
statistic,p_value =  stats.mannwhitneyu(df_commute[df_commute['wait_time']=='2 mins']['cost_per_ride'], df_commute[df_commute['wait_time']=='5 mins']['cost_per_ride'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))

Statistics = 65.000, P-value=0.137


### Average Express pool 

In [18]:
statistic,p_value =  stats.mannwhitneyu(df_commute[df_commute['wait_time']=='2 mins']['trips_express'], df_commute[df_commute['wait_time']=='5 mins']['trips_express'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))

Statistics = 68.000, P-value=0.093


# Non-Commute Hours

In [24]:
df_non_commute = df_raw[df_raw['commute']==False]

In [20]:
summary = df_raw[df_raw['commute']==False].groupby(['wait_time'])['rider_cancellations','trips_pool','trips_express','total_ride','cost_per_ride','match_rate','double_matches_rate'].agg('median').T
summary['diff (5mins - 2mins)'] = summary['5 mins'] - summary['2 mins']
summary.round(2)

wait_time,2 mins,5 mins,diff (5mins - 2mins)
rider_cancellations,156.00,170.00,14.00
trips_pool,1302.00,1425.00,123.00
trips_express,2453.00,2276.00,-177.00
total_ride,3729.00,3709.00,-20.00
cost_per_ride,7.23,6.88,-0.34
match_rate,0.64,0.62,-0.03
double_matches_rate,0.31,0.34,0.02


### Average Ride Cancellation

In [26]:
statistic,p_value =  stats.mannwhitneyu(df_non_commute[df_non_commute['wait_time']=='2 mins']['rider_cancellations'], df_non_commute[df_non_commute['wait_time']=='5 mins']['rider_cancellations'], alternative='less')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))

Statistics = 841.500, P-value=0.000


### Cost per trip

In [27]:
statistic,p_value =  stats.mannwhitneyu(df_non_commute[df_non_commute['wait_time']=='2 mins']['cost_per_ride'], df_non_commute[df_non_commute['wait_time']=='5 mins']['cost_per_ride'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))

Statistics = 1935.000, P-value=0.000


### Average Express pool 

In [28]:
statistic,p_value =  stats.mannwhitneyu(df_non_commute[df_non_commute['wait_time']=='2 mins']['trips_express'], df_non_commute[df_non_commute['wait_time']=='5 mins']['trips_express'], alternative='greater')
print('Statistics = %.3f, P-value=%.3f' %(statistic,p_value))

Statistics = 1838.500, P-value=0.003
